In [ ]:
!pip install fastapi uvicorn nest-asyncio

In [ ]:
from fastapi import FastAPI
from fastapi.staticfiles import StaticFiles


app = FastAPI()

app.mount("/", StaticFiles(directory="static", html=True), name="static")


output.serve_kernel_port_as_window(8000)
import uvicorn

uvicorn.run(app, port=8000, host="0.0.0.0")